In [9]:
import math
import numpy as np

try:
    import numba
    from numba import njit
    HAS_NUMBA = True
except Exception:
    HAS_NUMBA = False

def _estimate_numpy_batch(n: int, rng: np.random.Generator, tol: float = 1e-12) -> int:
    """Return count of 'hits' in a batch of n samples (NumPy vectorized)."""
    bx, by = rng.random(n), rng.random(n)
    rx, ry = rng.random(n), rng.random(n)

    # Closest side to Blue: 0=bottom(y=0), 1=top(y=1), 2=left(x=0), 3=right(x=1)
    dists = np.stack([by, 1.0 - by, bx, 1.0 - bx], axis=1)  # shape (n, 4)
    sides = dists.argmin(axis=1)

    # Perpendicular bisector of segment BR: a*x + b*y = c
    a = rx - bx
    b = ry - by
    c = 0.5 * (rx * rx + ry * ry - bx * bx - by * by)

    hits = np.zeros(n, dtype=bool)

    # y = 0 (bottom): a*x = c -> x = c/a
    m = (sides == 0)
    good = np.abs(a[m]) > tol
    x = np.empty(np.count_nonzero(m))
    x[:] = np.nan
    x[good] = c[m][good] / a[m][good]
    hits[m] = good & (x >= -tol) & (x <= 1.0 + tol)

    # y = 1 (top): a*x + b = c -> x = (c - b)/a
    m = (sides == 1)
    good = np.abs(a[m]) > tol
    x = np.empty(np.count_nonzero(m))
    x[:] = np.nan
    x[good] = (c[m][good] - b[m][good]) / a[m][good]
    hits[m] = good & (x >= -tol) & (x <= 1.0 + tol)

    # x = 0 (left): b*y = c -> y = c/b
    m = (sides == 2)
    good = np.abs(b[m]) > tol
    y = np.empty(np.count_nonzero(m))
    y[:] = np.nan
    y[good] = c[m][good] / b[m][good]
    hits[m] = good & (y >= -tol) & (y <= 1.0 + tol)

    # x = 1 (right): a + b*y = c -> y = (c - a)/b
    m = (sides == 3)
    good = np.abs(b[m]) > tol
    y = np.empty(np.count_nonzero(m))
    y[:] = np.nan
    y[good] = (c[m][good] - a[m][good]) / b[m][good]
    hits[m] = good & (y >= -tol) & (y <= 1.0 + tol)

    return int(hits.sum())


if HAS_NUMBA:
    @njit(cache=True, fastmath=True)
    def _estimate_numba_batch(bx, by, rx, ry, tol=1e-12):
        n = bx.size
        hits = 0
        for i in range(n):
            # closest side to blue
            d0, d1, d2, d3 = by[i], 1.0 - by[i], bx[i], 1.0 - bx[i]
            side = 0
            mind = d0
            if d1 < mind:
                side, mind = 1, d1
            if d2 < mind:
                side, mind = 2, d2
            if d3 < mind:
                side, mind = 3, d3

            a = rx[i] - bx[i]
            b = ry[i] - by[i]
            c = 0.5 * (rx[i]*rx[i] + ry[i]*ry[i] - bx[i]*bx[i] - by[i]*by[i])

            if side == 0:          # y = 0: a*x = c
                if abs(a) > tol:
                    x = c / a
                    if -tol <= x <= 1.0 + tol:
                        hits += 1
            elif side == 1:        # y = 1: a*x + b = c -> x = (c - b)/a
                if abs(a) > tol:
                    x = (c - b) / a
                    if -tol <= x <= 1.0 + tol:
                        hits += 1
            elif side == 2:        # x = 0: b*y = c -> y = c/b
                if abs(b) > tol:
                    y = c / b
                    if -tol <= y <= 1.0 + tol:
                        hits += 1
            else:                   # x = 1: a + b*y = c -> y = (c - a)/b
                if abs(b) > tol:
                    y = (c - a) / b
                    if -tol <= y <= 1.0 + tol:
                        hits += 1
        return hits


def run_mc_until_precision(batch_size=500_000,
                           max_samples=50_000_000,
                           ci_half_width_target=2e-4,
                           seed=42,
                           use_numba=True,
                           record_path=None):
    """
    Runs Monte Carlo in batches until the 95% CI half-width <= ci_half_width_target
    or until max_samples is reached. Returns (p_hat, n_total, (ci_lo, ci_hi), history).

    history is a list of (n_total, p_hat) pairs (useful for plotting convergence).
    """
    rng = np.random.default_rng(seed)
    total_hits = 0
    total_n = 0
    history = []

    while total_n < max_samples:
        n = batch_size

        if use_numba and HAS_NUMBA:
            bx, by = rng.random(n), rng.random(n)
            rx, ry = rng.random(n), rng.random(n)
            hits = _estimate_numba_batch(bx, by, rx, ry)
        else:
            hits = _estimate_numpy_batch(n, rng)

        total_hits += hits
        total_n += n

        p_hat = total_hits / total_n
        # normal approx CI (good for large n)
        se = math.sqrt(max(p_hat * (1 - p_hat), 1e-18) / total_n)
        ci_lo, ci_hi = p_hat - 1.96 * se, p_hat + 1.96 * se
        half_width = 1.96 * se

        history.append((total_n, p_hat))
        print(f"n={total_n:,}  p̂={p_hat:.10f}  95% CI=[{ci_lo:.10f}, {ci_hi:.10f}]  half-width={half_width:.2e}")

        if half_width <= ci_half_width_target:
            break

    if record_path:
        # Save convergence history to CSV (n, p_hat)
        import pandas as pd
        df = pd.DataFrame(history, columns=["n", "p_hat"])
        df.to_csv(record_path, index=False)

    return p_hat, total_n, (ci_lo, ci_hi), history


In [11]:
p_hat, n_total, ci, hist = run_mc_until_precision(
    batch_size=1_000_000,       # increase for faster convergence per print
    max_samples=80_000_000,     # hard cap
    ci_half_width_target=2e-4,  # smaller => tighter precision
    seed=123,
    use_numba=True,             # requires numba; will auto-fallback if missing
    record_path=None            # or e.g., "convergence.csv"
)

print("\nFinal:")
print(f"n_total = {n_total:,}")
print(f"p_hat   = {p_hat:.12f}")
print(f"95% CI  = [{ci[0]:.12f}, {ci[1]:.12f}]")


n=1,000,000  p̂=0.4912970000  95% CI=[0.4903171485, 0.4922768515]  half-width=9.80e-04
n=2,000,000  p̂=0.4911490000  95% CI=[0.4904561439, 0.4918418561]  half-width=6.93e-04
n=3,000,000  p̂=0.4911790000  95% CI=[0.4906132848, 0.4917447152]  half-width=5.66e-04
n=4,000,000  p̂=0.4913295000  95% CI=[0.4908395737, 0.4918194263]  half-width=4.90e-04
n=5,000,000  p̂=0.4913876000  95% CI=[0.4909493957, 0.4918258043]  half-width=4.38e-04
n=6,000,000  p̂=0.4913858333  95% CI=[0.4909858094, 0.4917858573]  half-width=4.00e-04
n=7,000,000  p̂=0.4914042857  95% CI=[0.4910339353, 0.4917746362]  half-width=3.70e-04
n=8,000,000  p̂=0.4913688750  95% CI=[0.4910224443, 0.4917153057]  half-width=3.46e-04
n=9,000,000  p̂=0.4913835556  95% CI=[0.4910569374, 0.4917101737]  half-width=3.27e-04
n=10,000,000  p̂=0.4913532000  95% CI=[0.4910433431, 0.4916630569]  half-width=3.10e-04
n=11,000,000  p̂=0.4913611818  95% CI=[0.4910657448, 0.4916566188]  half-width=2.95e-04
n=12,000,000  p̂=0.4913213333  95% CI=[0.